СОРТИРОВКА mask<br>
ГРУППИРОВКА <br>
СВОДНЫЕ ТАБЛИЦЫ <br>
СКЛЕИВАНИЕ ТАБЛИЦ <br>
ОБЪЕДИНЕНИЕ ТАБЛИЦ <br>

In [1]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_fe.csv', sep=',')
#melb_data.head()
#display(melb_df.info())

melb_df = melb_data.copy()#создание копии
#melb_df.head()

In [22]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
#display(melb_df['Date'])
quarter_sold = melb_df['Date'].dt.quarter
display(quarter_sold.value_counts())


3    4873
2    4359
4    2329
1    2019
Name: Date, dtype: int64

In [26]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
#display(melb_df.info())

СОРТИРОВКА

In [32]:
#melb_df.sort_values(by='Price').head(10) #повозрастанию, индексы не изменились
#melb_df.sort_values(by='Date', ascending=False) #по убыванию


melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']] # по возрастанию СНАЧАЛА расстояние,
# а ПОТОМ,внутри,цена,вывод каждые десятый


,Distance,Price
11428,0.0,387000.0
10512,0.7,600000.0
5727,1.2,485000.0
8671,1.2,595000.0
5736,1.2,740000.0
...,...,...
12011,38.0,680000.0
10673,38.0,810000.0
13429,38.0,1155000.0
11102,41.0,650000.0


In [33]:
#таунхаусах (Type), проданных компанией (SellerG) McGrath, у которых коэффициент соотношения
# площадей здания и участка (AreaRatio) меньше -0.8. Результат отсортируем по дате продажи
# (Date) в порядке возрастания, а после проведём сортировку по
# убыванию коэффициента соотношения площадей

   #выводим только два столбцаmask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],     #сортируем два столбца
    ascending=[True, False],      #дату по возрастанию,коэффициент площади по убыванию
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]  

,Date,AreaRatio
0,2016-07-26,-0.974922
1,2016-09-24,-0.971831
2,2016-11-27,-0.953608
3,2016-12-11,-0.945946
4,2017-08-04,-0.947368
5,2017-08-04,-0.970874


**ГРУППИРОВКА**

In [5]:
melb_df.groupby(by='Type').mean() #по всем столбцам

melb_df.groupby('Type')['Price'].mean() #средняя цена по типу, по одному столбцу цена (Series)

C:\Users\sveta\AppData\Local\Temp\ipykernel_7836\4068738370.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  melb_df.groupby(by='Type').mean() #по всем столбцам


Type
house        1.242665e+06
townhouse    9.337351e+05
unit         6.051275e+05
Name: Price, dtype: float64

In [6]:
# какие регионы (Regionname) наиболее удалены от центра Мельбурна
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

#наиболее удалёнными являются все регионы Victoria


Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

ГРУППИРОВКА

Построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale). Результат отсортируем по количеству продаж в порядке убывания

In [8]:
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)   #DataFrame



melb_df.groupby('MonthSale')['Price'].agg('describe')

,count,mean,std,min,25%,50%,75%,max
MonthSale,,,,,,,,
1,278.0,9.397921e+05,577668.924214,170000.0,570500.0,795000.0,1111250.0,5200000.0
2,333.0,1.169051e+06,671564.357417,131000.0,710000.0,1020000.0,1478000.0,4735000.0
3,1408.0,1.146762e+06,709573.596867,85000.0,680000.0,945000.0,1400000.0,5600000.0
4,1246.0,1.050479e+06,591892.902979,145000.0,655000.0,905500.0,1298750.0,5500000.0
5,1644.0,1.097807e+06,668492.867996,145000.0,650000.0,905000.0,1371250.0,8000000.0
6,1469.0,1.068981e+06,606010.069052,222000.0,660000.0,900000.0,1325000.0,7650000.0
7,1835.0,9.314698e+05,537390.803161,190000.0,586750.0,800000.0,1150000.0,9000000.0
8,1850.0,1.056371e+06,619617.476541,160000.0,635000.0,892000.0,1310000.0,6500000.0
9,1188.0,1.126349e+06,608734.690742,170000.0,725000.0,980000.0,1360000.0,6400000.0


In [9]:
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)
#получить множество из агентств недвижимости, которые работают в каждом из регионов

,nunique,set
Regionname,,
Eastern Metropolitan,26,"{Ray, Love, Buxton, hockingstuart, RT, Kay, No..."
Eastern Victoria,11,"{hockingstuart, Barry, Ray, Fletchers, McGrath..."
Northern Metropolitan,40,"{Nick, Village, Ray, Love, Sweeney, Raine, Ren..."
Northern Victoria,11,"{hockingstuart, LITTLE, YPA, Barry, Ray, Bucki..."
South-Eastern Metropolitan,25,"{Ray, Buxton, hockingstuart, Noel, Greg, O'Bri..."
Southern Metropolitan,38,"{Nick, Ray, Raine, Buxton, Rendina, hockingstu..."
Western Metropolitan,34,"{Moonee, Village, Ray, Sweeney, Raine, Rendina..."
Western Victoria,6,"{hockingstuart, YPA, Ray, other, Raine, HAR}"


СВОДНЫЕ ТАБЛИЦЫ

In [9]:
melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median()
melb_df.groupby(['Rooms', 'Type'])['Price'].mean()

melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack() #позволяет переопределить вложенный индекс в виде столбцов таблицы


melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()


# проанализируем продажи в каждом из регионов в зависимости от того,
# будний был день или выходной.
# таблицу, в которой строками будут являться названия регионов (Regionname),
# а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend),
# который равен 1, если день был выходным, и 0 — в противном случае.
# В качестве значений сводной таблицы возьмём количество продаж.
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)


melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

#по индексам будут располагаться признаки метода продажи (Method) и
# типа объекта (Type), по столбцам — наименование региона (Regionname),
# а на пересечении строк и столбцов будет стоять медианная цена объекта (Price)
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)


Regionname        Eastern Metropolitan  Eastern Victoria  \
Method Type                                                
PI     house                   1244000            780000   
       townhouse                760000                 0   
       unit                     650000                 0   
S      house                   1127000            675000   
       townhouse                828000                 0   
       unit                     645750            492000   
SA     house                    932500            950000   
       townhouse                807500                 0   
       unit                          0                 0   
SP     house                   1050000            672500   
       townhouse                910000                 0   
       unit                     515000            400000   
VB     house                   1100000            712500   
       townhouse                892500                 0   
       unit                     500000                 0   

Regionname        Northern Metropolitan  Northern Victoria  \
Method Type                                                  
PI     house                     900000             500000   
       townhouse                 632500                  0   
       unit                      410000                  0   
S      house                     920000             555000   
       townhouse                 750000                  0   
       unit                      525500                  0   
SA     house                     817500             540000   
       townhouse                 425000                  0   
       unit                      616000                  0   
SP     house                     900000             521000   
       townhouse                 690000                  0   
       unit                      470000                  0   
VB     house                    1050000             690000   
       townhouse                 640000                  0   
       unit                      450000                  0   

Regionname        South-Eastern Metropolitan  Southern Metropolitan  \
Method Type                                                           
PI     house                          865000                1725000   
       townhouse                     1190000                1055000   
       unit                           525000                 571250   
S      house                          883300                1611000   
       townhouse                      875000                1135000   
       unit                           606000                 655000   
SA     house                          880000                1390000   
       townhouse                           0                1141000   
       unit                                0                 580000   
SP     house                          770000                1521750   
       townhouse                      800000                1162500   
       unit                           601000                 550000   
VB     house                          850000                1800000   
       townhouse                           0                1250000   
       unit                           700000                 500000   

Regionname        Western Metropolitan  Western Victoria  
Method Type                                               
PI     house                    870000            630000  
       townhouse                670000                 0  
       unit                     360000                 0  
S      house                    870000            397500  
       townhouse                729000                 0  
       unit                     489000                 0  
SA     house                    772500                 0  
       townhouse                467500                 0  
       unit                     571000                 0  
SP     house                    865000            360000  
       t

In [14]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
pivot.columns #получаем объект MultiIndex

#доступ
display(pivot['mean']['unit'])

#фильтрация
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

print(list(filtered_pivot.index))

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

median                       mean              \
Type                   house townhouse unit       house   townhouse   
Regionname                                                            
Southern Metropolitan  586.0     246.0    0  569.643881  278.858824   
Western Metropolitan   531.0     198.0   62  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

['Southern Metropolitan', 'Western Metropolitan']


In [16]:
import numpy as np
mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
display(mser)

print(mser.index)



mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

white  up       0.014961
       down     0.782687
       right    0.009016
blue   up       0.066805
       down     0.756409
red    up       0.186699
       down     0.486681
       left     0.080769
dtype: float64

MultiIndex([('white',    'up'),
            ('white',  'down'),
            ('white', 'right'),
            ( 'blue',    'up'),
            ( 'blue',  'down'),
            (  'red',    'up'),
            (  'red',  'down'),
            (  'red',  'left')],
           )


pen               paper          
                   1         2         1         2
white up   -0.828925  0.018973 -0.519152 -0.944961
      down  0.025389  0.079607 -0.183554 -0.787427
red   up    0.853233  2.094381 -0.879914 -1.491398
      down -1.288910 -0.365957  0.623098 -0.720431

MovieLens

In [26]:
ratings1 = pd.read_csv('data/ratings1.csv', sep=',')
ratings2 = pd.read_csv('data/ratings2.csv', sep=',')
movies = pd.read_csv('data/movies.csv', sep=',')
dates = pd.read_csv('data/dates.csv', sep=',')

ratings1_df = ratings1.copy()
ratings2_df = ratings2.copy()
movies_df = movies.copy()
dates_df = dates.copy()

ratings1_df.info() #уникальных фильмов представлено в таблице movies

ratings1_df['userId'].nunique() #Сколько уникальных пользователей в таблице ratings1

dates_df['date'] = pd.to_datetime(dates_df['date'])
display(dates_df['date'].dt.year.mode())  #каком году было выставлено больше всего оценок



0    2000
Name: date, dtype: int64

Склеивание таблиц

In [34]:
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
#display(ratings)

#аналогичный метод  АВОМАТИЧЕСКИ УДАЛЯЕТ ПОВТОРЫ!!!!!
merge_ratings = ratings1.merge(ratings2, how='outer')
print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
display(merge_ratings)


print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

#дублирование строк
display(ratings1.tail(1))
display(ratings2.head(1))

ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])
# Число строк в таблице ratings: 100836

#конкатенируем таблицы ratings и dates
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))

Число строк в таблице ratings:  100837
Число строк в таблице dates:  100836
False


,userId,movieId,rating
40000,274,5621,2.0


,userId,movieId,rating
0,274,5621,2.0


Число строк в таблице ratings:  100836


,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


Объединение таблиц

In [39]:
#первый способ join()
joined = ratings_dates.join(
    movies.set_index('movieId'),   #set_index()
    on='movieId',
    how='left'
)
#display(joined.head())

#второй способ merge()
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
#display(merged.head())

#проверяем что таблицы получились одинаковые
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])

Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True


In [ ]:
#такоф тип full не подхоит, проверяем хвостом таблицы
merged2 = ratings_dates.merge(
    movies,
    on='movieId',
    how='outer'
)
print('Число строк в таблице merged2: ', merged2.shape[0])
display(merged2.tail())